<a href="https://colab.research.google.com/github/Antsistra/Rute-Rumah-Sakit-Terdekat-A-Stars/blob/dev-models/Rute_Rumah_sakit_Terdekat_dari_Perumahan_X_Menggunakan_A_(A_Stars)_Method_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Penerapan Algoritma A* (A-Star) dalam Mencari Rute Terpendek Dari Citraland ke Rumah Sakit Sari Asih Serang**


**Import Library**

In [ ]:
import folium as fl
import pandas as pd
import numpy as np
import heapq

# **Dataset Section**

**Generate Dataset**

Generate Dataset For Eculidean Calculation

In [ ]:

data = {
    "No": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    "Titik": ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L"],
    "Lokasi": [
        "Citraland", "Jl Raya Sepang", "Jl Ciracas", "Jl Kolonel Tubagus Suwandi",
        "Jl Veteran", "Jl Kh Abdul Hadi", "Jl Kejaksaan I", "Jl Jend Ahmad Yani",
        "Jl Kh Sochan", "Jl Kh abdul fatah hasan", "Jl Jend Sudirman", "Rs Sari Asih Serang"
    ],
    "Koordinat": [
        "0, 0", "1, 0", "4, 4",
        "5, 5", "6, 13", "8, 8",
        "8, 10", "9, 13", "12, 12",
        "15, 12", "17, 16", "18, 17"
    ]
}


data['X'] = [float(coord.split(", ")[0]) for coord in data['Koordinat']]
data['Y'] = [float(coord.split(", ")[1]) for coord in data['Koordinat']]


df = pd.DataFrame(data)
df.to_csv("dataset/koordinat.csv", index=False)
dataset = pd.read_csv('dataset/koordinat.csv')
dataset


Cost Value Dataset f(n)

In [ ]:
data = {
    "No": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
    "Titik": ["AB", "BC", "BD", "CE","DF","FG","FI","FJ","GE","EH","HK","JK","IK","KL"],
    "f(n)": [0.95,1.4,1.5,1.9,1.6,0.5,0.7,1.4,1.2,0.6,1.6,0.7,1.3,1],

}


df = pd.DataFrame(data)

df.to_csv("dataset/cost_value.csv", index=False)
dataset = pd.read_csv('dataset/cost_value.csv')
dataset

Visualisasi Lokasi

In [ ]:

data = {
    "No": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    "Titik": ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L"],
    "Lokasi": [
        "Citraland", "Jl Raya Sepang", "Jl Ciracas", "Jl Kolonel Tubagus Suwandi",
        "Jl Veteran", "Jl Kh Abdul Hadi", "Jl Kejaksaan I", "Jl Jend Ahmad Yani",
        "Jl Kh Sochan", "Jl Kh abdul fatah hasan", "Jl Jend Sudirman", "Rs Sari Asih Serang"
    ],
    "Koordinat": [
        "-6.131067, 106.132513", "-6.133913, 106.134790", "-6.127008, 106.144934",
        "-6.127864, 106.146556", "-6.116890, 106.153907", "-6.125760, 106.159867",
        "-6.123619, 106.156434", "-6.118500, 106.159189", "-6.122556, 106.164502",
        "-6.124513, 106.171925", "-6.120478, 106.173673", "-6.118677, 106.181270"
    ]
}


data['X'] = [float(coord.split(", ")[0]) for coord in data['Koordinat']]
data['Y'] = [float(coord.split(", ")[1]) for coord in data['Koordinat']]


df = pd.DataFrame(data)
df.to_csv("dataset/location.csv", index=False)
dataset = pd.read_csv('dataset/location.csv')
dataset


Menampilkan Lokasi dan Map

In [ ]:

_map = fl.Map(location=[-6.1236296, 106.1556118], zoom_start=15.4)


for index, row in dataset.iterrows():
    fl.Marker(
        location=[row['X'], row['Y']],
        popup=row['Lokasi'],
        tooltip=row['Lokasi'],
    ).add_to(_map)
    fl.Marker(location=[-6.131067,106.132513], icon=fl.Icon(color='green',icon='home',prefix='fa')).add_to(_map)
    fl.Marker(location=[-6.118677,106.181270], icon=fl.Icon(color='red',icon='plus-square',prefix='fa')).add_to(_map)

_map

Data Jarak Sesungguhnya g(n)


Implementasi Perhitungan euclidean  untuk menentukan h(n)


In [ ]:
file_path = 'dataset/koordinat.csv'
data = pd.read_csv(file_path)

nodes = data['Titik'].unique()

coordinates = {row['Titik']: (row['X'], row['Y']) for _, row in data.iterrows()}

def euclidean_distance(point1, point2):
    x1, y1 = coordinates[point1]
    x2, y2 = coordinates[point2]
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

goal_node = 'L'

heuristic = []
for node in nodes:
    distance = euclidean_distance(node, goal_node)
    heuristic.append((node, distance))

results_df = pd.DataFrame(heuristic, columns=['Titik', 'nilai_heuristic'])
results_df.to_csv('dataset/euclidean.csv', index=False)

In [ ]:
dataset = pd.read_csv('dataset/euclidean.csv')
dataset

# **Implementasi Algoritma A***

In [ ]:
def astar(start, goal, graph, heuristic):
    open_set = [(0, start)]
    close_set = set()
    g_cost = {start: 0}
    parent = {start: None}

    while open_set:

        current_cost, current_node = heapq.heappop(open_set)


        if current_node == goal:
            path = []
            total_distance = 0
            while current_node is not None:
                path.append(current_node)
                if parent[current_node] is not None:
                    total_distance += graph[current_node][parent[current_node]]
                current_node = parent[current_node]
            path.reverse()
            return path, total_distance

        close_set.add(current_node)


        for neighbor in graph.get(current_node, []):
            if neighbor in close_set:
                continue


            new_g_cost = g_cost[current_node] + graph[current_node][neighbor]

            if neighbor not in [node[1] for node in open_set] or new_g_cost < g_cost.get(neighbor, float('inf')):
                g_cost[neighbor] = new_g_cost
                f_cost = new_g_cost + heuristic[neighbor]
                heapq.heappush(open_set, (f_cost, neighbor))
                parent[neighbor] = current_node

    return None, None

cost_data = pd.read_csv('dataset/cost_value.csv')
graph = {}
for _, row in cost_data.iterrows():
    node1, node2 = row['Titik']
    cost = row['f(n)']
    if node1 not in graph:
        graph[node1] = {}
    if node2 not in graph:
        graph[node2] = {}
    graph[node1][node2] = cost
    graph[node2][node1] = cost

heuristic_data = pd.read_csv('dataset/euclidean.csv')
heuristic = dict(zip(heuristic_data['Titik'], heuristic_data['nilai_heuristic']))

start = 'A'
goal = 'L'
path, total_distance = astar(start, goal, graph, heuristic)

if path:
    print(f"Jalur terdekat dari {start} ke {goal}: {' -> '.join(path)}")
    print(f"Total jarak sesungguhnya: {total_distance} Km")
else:
    print(f"Tidak ditemukan jalur dari {start} ke {goal}")

**Visualisasi Jalur Terpendek**

In [ ]:

data = {
    "No": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    "Titik": ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L"],
    "Lokasi": [
        "Citraland", "Jl Raya Sepang", "Jl Ciracas", "Jl Kolonel Tubagus Suwandi",
        "Jl Veteran", "Jl Kh Abdul Hadi", "Jl Kejaksaan I", "Jl Jend Ahmad Yani",
        "Jl Kh Sochan", "Jl Kh abdul fatah hasan", "Jl Jend Sudirman", "Rs Sari Asih Serang"
    ],
    "Koordinat": [
        "-6.131067, 106.132513", "-6.133913, 106.134790", "-6.127008, 106.144934",
        "-6.127864, 106.146556", "-6.116890, 106.153907", "-6.125760, 106.159867",
        "-6.123619, 106.156434", "-6.118500, 106.159189", "-6.122556, 106.164502",
        "-6.124513, 106.171925", "-6.120478, 106.173673", "-6.118677, 106.181270"
    ]
}

data['X'] = [float(coord.split(", ")[0]) for coord in data['Koordinat']]
data['Y'] = [float(coord.split(", ")[1]) for coord in data['Koordinat']]

df = pd.DataFrame(data)


_map = fl.Map(location=[-6.1236296, 106.1556118], zoom_start=15.4)

for index, row in df.iterrows():
    fl.Marker(
        location=[row['X'], row['Y']],
        popup=row['Lokasi'],
        tooltip=row['Lokasi'],
    ).add_to(_map)

fl.Marker(location=[-6.131067, 106.132513], icon=fl.Icon(color='green', icon='home', prefix='fa')).add_to(_map)
fl.Marker(location=[-6.118677, 106.181270], icon=fl.Icon(color='red', icon='plus-square', prefix='fa')).add_to(_map)


coordinates = df[df['Titik'].isin(path)][['X', 'Y']].values.tolist()
fl.PolyLine(locations=coordinates, color='blue', weight=5).add_to(_map)

_map